In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pickle

import janestreet

from sklearn import linear_model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv'
                    #, nrows = 1000000
                    )

In [ ]:
features = [c for c in train.columns if 'feature' in c]
f_na_num = np.isnan(train[features]).sum()
f_mean = train[features].mean()
train[features] = train[features].fillna(f_mean)


In [ ]:
regr = linear_model.LinearRegression()
regr.fit(train[features], train["resp"])
print(regr.score(train[features], train["resp"]))

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()
f_mean = f_mean.values

In [ ]:
weight_mid = train["weight"].quantile(0.5)
resp_mid = train["resp"].quantile(0.5)
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        weight = test_df.loc[:, "weight"].values
        if np.isnan(x_tt[:, 0:].sum()):
            x_tt[:, 0:] = np.nan_to_num(x_tt[:, 0:]) + np.isnan(x_tt[:, 0:]) * f_mean
        pred = 0.
        ret = regr.predict(x_tt)
        pred_df.action = 1 if (ret > resp_mid or weight < weight_mid) else 0
    env.predict(pred_df)